<a href="https://colab.research.google.com/github/FtM1997/Alzheimer-s-disease-Paper/blob/main/Fusion_Alzheimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries & models



In [ ]:
import os
import numpy as np
import math
import random
from collections import Counter
from google.colab import files
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, Model, clone_model
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, ConvLSTM2D,MaxPooling3D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, EfficientNetB2
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Input
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import shutil


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/'

def extract_numeric_part(folder_name):
    parts = folder_name.split('_')
    for part in parts:
        if part.startswith('subject'):
            return int(part.replace('subject', ''))
    return -1  # Return -1 if no numeric part is found

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

subject_folders = np.array(subject_folders)
labels = np.array(labels)

print("Subject Folders:", subject_folders)
print("Labels:", labels)

Subject Folders: ['/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject1'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject2'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject3'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject4'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject5'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject6'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject7'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject8'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject9'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/sub

In [ ]:
def load_images_and_labels(subjects, labels):
    images, labels_t = [], []
    for subject_folder, label in zip(subjects, labels):
        for filename in os.listdir(subject_folder):
            img_path = os.path.join(subject_folder, filename.decode())
            try:
                img = Image.open(img_path)

                width, height = img.size
                left = 100
                top = 70
                right = width - 200
                bottom = height - 65
                img = img.crop((left, top, right, bottom))

                img = img.resize((299, 299))
                img_array = img_to_array(img)
                img_array = img_array / 255.0

                images.append(img_array)
                labels_t.append(label)
            except Exception as e:
                print(f"Error loading image: {img_path}, {e}")

    return np.array(images), np.array(labels_t)


In [ ]:
# def load_and_preprocess_data(subject_folders, labels):
#     train_subjects, test_subjects, train_labels, test_labels = train_test_split(subject_folders, labels, test_size=0.2, stratify=labels)

#     train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, test_size=0.1, stratify=train_labels)

#     train_images, train_labels_t = load_images_and_labels(train_subjects, train_labels)
#     val_images, val_labels_t = load_images_and_labels(val_subjects, val_labels)

#     test_image_counts_per_subject = [len(os.listdir(folder)) for folder in test_subjects]

#     test_images, test_labels_t = load_images_and_labels(test_subjects, test_labels)

#     return train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject

def load_and_preprocess_data(subject_folders, labels):
    # Create dictionaries to categorize subjects into train, val, and test groups
    train_subjects_dict = {}
    val_subjects_dict = {}
    test_subjects_dict = {}

    for subject_folder, label in zip(subject_folders, labels):
        subject_num = extract_numeric_part(os.path.basename(subject_folder))
        if 3 <= subject_num <= 5 or 45 <= subject_num <= 47:
            val_subjects_dict[subject_folder] = label
        elif 6 <= subject_num <= 12 or 48 <= subject_num <= 53:
            test_subjects_dict[subject_folder] = label
        else:
            train_subjects_dict[subject_folder] = label

    # Extract subjects and labels for each group
    train_subjects = list(train_subjects_dict.keys())
    train_labels = [train_subjects_dict[subj] for subj in train_subjects]
    val_subjects = list(val_subjects_dict.keys())
    val_labels = [val_subjects_dict[subj] for subj in val_subjects]
    test_subjects = list(test_subjects_dict.keys())
    test_labels = [test_subjects_dict[subj] for subj in test_subjects]

    # Load images for each group
    train_images, train_labels_t = load_images_and_labels(train_subjects, train_labels)
    val_images, val_labels_t = load_images_and_labels(val_subjects, val_labels)
    test_images, test_labels_t = load_images_and_labels(test_subjects, test_labels)

    return train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_images


In [ ]:
def create_new_model(base_model):
    model = clone_model(base_model)

    for layer in model.layers:
        layer.trainable = False

    x = model.output
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = Dropout(0.2)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=model.input, outputs=predictions)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [ ]:
# Define a custom callback to start saving checkpoints from epoch 30
class CustomCheckpoint(ModelCheckpoint):
    def __init__(self, start_epoch, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch >= self.start_epoch:
            super().on_epoch_end(epoch, logs)

def train_and_evaluate_model(base_model, subject_folders, labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")

        model = create_new_model(base_model)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(subject_folders, labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)

        checkpoint_dir = '/content'
        os.makedirs(checkpoint_dir, exist_ok=True)

        local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'

        checkpoint_callback = CustomCheckpoint(
            start_epoch=30,
            filepath=local_checkpoint_filepath,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )

        class LearningRateLogger(Callback):
            def on_epoch_end(self, epoch, logs=None):
                logs = logs or {}
                logs['learning_rate'] = self.model.optimizer.learning_rate.numpy()
        lr_logger = LearningRateLogger()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False, start_from_epoch=30)

        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=32, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping, lr_logger], verbose=1)

        if os.path.exists(local_checkpoint_filepath):
            print(f"Best weights saved locally at: {local_checkpoint_filepath}")

            shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
            if os.path.exists(drive_checkpoint_filepath):
                print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
            else:
                print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        else:
            print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")

        model.load_weights(local_checkpoint_filepath)

        test_predictions = model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        print("ModelCheckpoint callback triggered and saved the best weights.")

        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)

        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx




    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")



# Train for Spectrogram on VGG, DenseNet, ResNet

## VGG16: 3 iterations

In [ ]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
model_name = 'vgg16_Central_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)


Iteration 1:
Epoch 1/100
169/169 [==============================] - 117s 683ms/step - loss: 0.6969 - accuracy: 0.5299 - val_loss: 0.6889 - val_accuracy: 0.5814 - learning_rate: 0.0010
Epoch 2/100
169/169 [==============================] - 112s 662ms/step - loss: 0.6935 - accuracy: 0.5347 - val_loss: 0.7123 - val_accuracy: 0.4186 - learning_rate: 0.0010
Epoch 3/100
169/169 [==============================] - 112s 662ms/step - loss: 0.6895 - accuracy: 0.5544 - val_loss: 0.7023 - val_accuracy: 0.4186 - learning_rate: 0.0010
Epoch 4/100
169/169 [==============================] - 111s 658ms/step - loss: 0.6873 - accuracy: 0.5518 - val_loss: 0.6977 - val_accuracy: 0.4186 - learning_rate: 0.0010
Epoch 5/100
169/169 [==============================] - 112s 665ms/step - loss: 0.6838 - accuracy: 0.5576 - val_loss: 0.6945 - val_accuracy: 0.4186 - learning_rate: 0.0010
Epoch 6/100
169/169 [==============================] - 111s 657ms/step - loss: 0.6608 - accuracy: 0.5918 - val_loss: 0.6806 - val_a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Predictions for Each Subject's Images:
5/5 [==============================] - 3s 531ms/step
Subject 1 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject7):
AD count: [144], HC count: [0]
7/7 [==============================] - 4s 590ms/step
Subject 2 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject44):
AD count: [29], HC count: [190]
4/4 [==============================] - 2s 579ms/step
Subject 3 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject9):
AD count: [15], HC count: [108]
4/4 [==============================] - 2s 531ms/step
Subject 4 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject37):
AD count: [31], HC count: [83]
5/5 [==============================] - 2s 471ms/step
Subject 5 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject17):
AD count: [129], HC count:

In [ ]:
model_name = 'vgg16_Central_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)


Iteration 1:
Epoch 1/100
176/176 [==============================] - 113s 635ms/step - loss: 0.6993 - accuracy: 0.5162 - val_loss: 0.7148 - val_accuracy: 0.4733 - learning_rate: 0.0010
Epoch 2/100
176/176 [==============================] - 109s 622ms/step - loss: 0.6815 - accuracy: 0.5652 - val_loss: 0.6824 - val_accuracy: 0.5281 - learning_rate: 0.0010
Epoch 3/100
176/176 [==============================] - 109s 621ms/step - loss: 0.6512 - accuracy: 0.6212 - val_loss: 0.6395 - val_accuracy: 0.6982 - learning_rate: 0.0010
Epoch 4/100
176/176 [==============================] - 110s 626ms/step - loss: 0.6110 - accuracy: 0.6667 - val_loss: 0.6161 - val_accuracy: 0.6543 - learning_rate: 0.0010
Epoch 5/100
176/176 [==============================] - 110s 625ms/step - loss: 0.5875 - accuracy: 0.6877 - val_loss: 0.6226 - val_accuracy: 0.6584 - learning_rate: 0.0010
Epoch 6/100
176/176 [==============================] - 109s 621ms/step - loss: 0.5391 - accuracy: 0.7237 - val_loss: 0.8167 - val_a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Predictions for Each Subject's Images:
2/2 [==============================] - 1s 307ms/step
Subject 1 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject61):
AD count: [32], HC count: [16]
3/3 [==============================] - 2s 505ms/step
Subject 2 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject20):
AD count: [89], HC count: [1]
4/4 [==============================] - 2s 413ms/step
Subject 3 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject53):
AD count: [2], HC count: [100]
6/6 [==============================] - 3s 487ms/step
Subject 4 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject34):
AD count: [90], HC count: [81]
3/3 [==============================] - 2s 502ms/step
Subject 5 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject51):
AD count: [0], HC count: [9

## ResNet50: 2

In [ ]:
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
model_name = 'ResNet50_Central_Spectrogram'
train_and_evaluate_model(resnet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)


Iteration 1:
Epoch 1/100
169/169 [==============================] - 113s 647ms/step - loss: 1.2143 - accuracy: 0.5119 - val_loss: 0.8040 - val_accuracy: 0.3606 - learning_rate: 0.0010
Epoch 2/100
169/169 [==============================] - 109s 645ms/step - loss: 0.9056 - accuracy: 0.5137 - val_loss: 0.8740 - val_accuracy: 0.3606 - learning_rate: 0.0010
Epoch 3/100
169/169 [==============================] - 110s 649ms/step - loss: 0.8341 - accuracy: 0.5317 - val_loss: 0.6372 - val_accuracy: 0.6394 - learning_rate: 0.0010
Epoch 4/100
169/169 [==============================] - 110s 651ms/step - loss: 0.7817 - accuracy: 0.5581 - val_loss: 0.6217 - val_accuracy: 0.6877 - learning_rate: 0.0010
Epoch 5/100
169/169 [==============================] - 108s 642ms/step - loss: 0.7043 - accuracy: 0.5924 - val_loss: 0.7620 - val_accuracy: 0.3606 - learning_rate: 0.0010
Epoch 6/100
169/169 [==============================] - 109s 643ms/step - loss: 0.6781 - accuracy: 0.6140 - val_loss: 0.8974 - val_a

## Densenet121: 3 iterations

In [ ]:
Dense_base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
model_name = 'Densenet121_Central_Spectrogram'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)


Iteration 1:
Epoch 1/100
182/182 [==============================] - 131s 682ms/step - loss: 0.7957 - accuracy: 0.5292 - val_loss: 1.2115 - val_accuracy: 0.4836 - learning_rate: 0.0010
Epoch 2/100
182/182 [==============================] - 122s 669ms/step - loss: 0.7042 - accuracy: 0.5965 - val_loss: 0.8075 - val_accuracy: 0.3365 - learning_rate: 0.0010
Epoch 3/100
182/182 [==============================] - 117s 640ms/step - loss: 0.6474 - accuracy: 0.6443 - val_loss: 1.0543 - val_accuracy: 0.4836 - learning_rate: 0.0010
Epoch 4/100
182/182 [==============================] - 121s 665ms/step - loss: 0.6069 - accuracy: 0.6732 - val_loss: 0.9023 - val_accuracy: 0.2895 - learning_rate: 0.0010
Epoch 5/100
182/182 [==============================] - 121s 667ms/step - loss: 0.5916 - accuracy: 0.6816 - val_loss: 0.9474 - val_accuracy: 0.3005 - learning_rate: 0.0010
Epoch 6/100
182/182 [==============================] - 112s 616ms/step - loss: 0.5796 - accuracy: 0.6963 - val_loss: 0.8599 - val_a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Predictions for Each Subject's Images:
2/2 [==============================] - 1s 698ms/step
Subject 1 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject4):
AD count: [63], HC count: [0]
4/4 [==============================] - 2s 462ms/step
Subject 2 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject41):
AD count: [0], HC count: [105]
3/3 [==============================] - 2s 534ms/step
Subject 3 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject20):
AD count: [86], HC count: [4]
5/5 [==============================] - 3s 537ms/step
Subject 4 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject7):
AD count: [131], HC count: [13]
3/3 [==============================] - 2s 497ms/step
Subject 5 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject3):
AD count: [36], HC count: [45]

In [ ]:
model_name = 'Densenet121_Central_Spectrogram'
train_and_evaluate_model(Dense_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)


Iteration 1:
Epoch 1/100
172/172 [==============================] - 125s 688ms/step - loss: 0.7272 - accuracy: 0.5824 - val_loss: 0.9178 - val_accuracy: 0.5862 - learning_rate: 0.0010
Epoch 2/100
172/172 [==============================] - 110s 641ms/step - loss: 0.6325 - accuracy: 0.6738 - val_loss: 0.7366 - val_accuracy: 0.4063 - learning_rate: 0.0010
Epoch 3/100
172/172 [==============================] - 107s 623ms/step - loss: 0.5508 - accuracy: 0.7327 - val_loss: 0.9936 - val_accuracy: 0.5723 - learning_rate: 0.0010
Epoch 4/100
172/172 [==============================] - 105s 612ms/step - loss: 0.5261 - accuracy: 0.7387 - val_loss: 0.6694 - val_accuracy: 0.6792 - learning_rate: 0.0010
Epoch 5/100
172/172 [==============================] - 103s 601ms/step - loss: 0.5292 - accuracy: 0.7354 - val_loss: 0.6527 - val_accuracy: 0.6314 - learning_rate: 0.0010
Epoch 6/100
172/172 [==============================] - 108s 628ms/step - loss: 0.4873 - accuracy: 0.7697 - val_loss: 0.7375 - val_a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Predictions for Each Subject's Images:
4/4 [==============================] - 2s 600ms/step
Subject 1 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject59):
AD count: [31], HC count: [95]
5/5 [==============================] - 3s 511ms/step
Subject 2 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject1):
AD count: [138], HC count: [0]
4/4 [==============================] - 2s 399ms/step
Subject 3 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject5):
AD count: [107], HC count: [1]
4/4 [==============================] - 2s 497ms/step
Subject 4 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject53):
AD count: [29], HC count: [73]
2/2 [==============================] - 1s 473ms/step
Subject 5 (/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Central/subject26):
AD count: [57], HC count: [0

# Inception V3 for Spectrogram & Scalogram

In [ ]:
Inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
model_name = 'InceptionV3_Temporal_Spectrogram'
train_and_evaluate_model(Inception_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:


ValueError: not enough values to unpack (expected 8, got 6)

In [ ]:
data_dir = '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/'

def extract_numeric_part(folder_name):
    return int(folder_name.split('_')[-1])

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

subject_folders = np.array(subject_folders)
labels = np.array(labels)

print("Subject Folders:", subject_folders)
print("Labels:", labels)

In [ ]:
Inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
model_name = 'InceptionV3_Temporal_Scalogram'
train_and_evaluate_model(Inception_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)

# Fusion Model

## Inception V3 for Spectrogram & Scalogram


### Spectrogram

In [ ]:
data_dir = '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/'

def extract_numeric_part(folder_name):
    parts = folder_name.split('_')
    for part in parts:
        if part.startswith('subject'):
            return int(part.replace('subject', ''))
    return -1  # Return -1 if no numeric part is found

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

Spectrogram_subject_folders, Spectrogram_labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    Spectrogram_subject_folder = os.path.join(data_dir, folder_name)
    Spectrogram_subject_folders.append(Spectrogram_subject_folder)

    if subject_num <= 36:
        Spectrogram_labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        Spectrogram_labels.append(1)  # HC group

Spectrogram_subject_folders = np.array(Spectrogram_subject_folders)
Spectrogram_labels = np.array(Spectrogram_labels)

print("Subject Folders:", Spectrogram_subject_folders)
print("Labels:", Spectrogram_labels)

Subject Folders: ['/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject1'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject2'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject3'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject4'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject5'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject6'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject7'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject8'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/subject9'
 '/content/drive/MyDrive/Alzheimer/Spectrogram Images/Spectrogram_Images_output/Temporal/sub

In [ ]:
import json
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from tensorflow.keras.models import model_from_json

# Define a custom callback to start saving checkpoints from epoch 30
class CustomCheckpoint(ModelCheckpoint):
    def __init__(self, start_epoch, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch >= self.start_epoch:
            super().on_epoch_end(epoch, logs)

def train_and_evaluate_model(base_model, Spectrogram_subject_folders, Spectrogram_labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")

        model = create_new_model(base_model)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(Spectrogram_subject_folders, Spectrogram_labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)

        checkpoint_dir = '/content'
        os.makedirs(checkpoint_dir, exist_ok=True)

        local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'

        checkpoint_callback = CustomCheckpoint(
            start_epoch=30,
            filepath=local_checkpoint_filepath,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )

        class LearningRateLogger(Callback):
            def on_epoch_end(self, epoch, logs=None):
                logs = logs or {}
                logs['learning_rate'] = self.model.optimizer.learning_rate.numpy()
        lr_logger = LearningRateLogger()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False, start_from_epoch=30)

        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=32, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping, lr_logger], verbose=1)

        # Save the model architecture as JSON
        model_json = model.to_json()
        json_filepath = os.path.join(checkpoint_dir, f"{model_name}_iteration_{iteration + 1}.json")
        with open(json_filepath, "w") as json_file:
            json_file.write(model_json)
        shutil.copy(json_filepath, f'/content/drive/My Drive/{model_name}_iteration_{iteration + 1}.json')

        # Save the model weights
        if os.path.exists(local_checkpoint_filepath):
            print(f"Best weights saved locally at: {local_checkpoint_filepath}")
            shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
            if os.path.exists(drive_checkpoint_filepath):
                print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
            else:
                print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        else:
            print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")

        # Load the model architecture and weights for evaluation
        with open(json_filepath, "r") as json_file:
            loaded_model_json = json_file.read()
        loaded_model = model_from_json(loaded_model_json)
        loaded_model.load_weights(local_checkpoint_filepath)

        test_predictions = loaded_model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)

        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (loaded_model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx

    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")


In [ ]:
Inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
model_name = 'InceptionV3_Temporal_Spectrogram'
train_and_evaluate_model(Inception_base_model, Spectrogram_subject_folders, Spectrogram_labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
239/239 [==============================] - 135s 545ms/step - loss: 0.7003 - accuracy: 0.5307 - val_loss: 0.6766 - val_accuracy: 0.5723 - learning_rate: 0.0010
Epoch 2/100
239/239 [==============================] - 126s 530ms/step - loss: 0.6885 - accuracy: 0.5422 - val_loss: 0.6542 - val_accuracy: 0.6210 - learning_rate: 0.0010
Epoch 3/100
239/239 [==============================] - 138s 580ms/step - loss: 0.6438 - accuracy: 0.6251 - val_loss: 0.6032 - val_accuracy: 0.6270 - learning_rate: 0.0010
Epoch 4/100
239/239 [==============================] - 141s 589ms/step - loss: 0.6229 - accuracy: 0.6604 - val_loss: 0.5717 - val_accuracy: 0.7199 - learning_rate: 0.0010
Epoch 5/100
239/239 [==============================] - 139s 583ms/step - loss: 0.5800 - accuracy: 0.7101 - val_loss: 0.6177 - val_accuracy: 0.6471 - learning_rate: 0.0010
Epoch 6/100
239/239 [==============================] - 141s 591ms/step - loss: 0.5462 - accuracy: 0.7353 - val_loss: 0.5854 - val_a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Predictions for Each Subject's Images:


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
# Function to save model architecture
def save_model_architecture(model, model_name):
    model_json = model.to_json()
    with open(f"{model_name}_architecture.json", "w") as json_file:
        json_file.write(model_json)

# Function to load model architecture
def load_model_architecture(model_name):
    with open(f"{model_name}_architecture.json", "r") as json_file:
        model_json = json_file.read()
    return model_from_json(model_json)

# Save base model architecture
base_model = create_new_model(base_model)
save_model_architecture(base_model, 'base_model')

# Load base model architecture
base_model = load_model_architecture('base_model')


### Scalogram

In [ ]:
data_dir = '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/'

def extract_numeric_part(folder_name):
    return int(folder_name.split('_')[-1])

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

Scalogram_subject_folders, Scalogram_labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    Scalogram_subject_folder = os.path.join(data_dir, folder_name)
    Scalogram_subject_folders.append(Scalogram_subject_folder)

    if subject_num <= 36:
        Scalogram_labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        Scalogram_labels.append(1)  # HC group

Scalogram_subject_folders = np.array(Scalogram_subject_folders)
Scalogram_labels = np.array(Scalogram_labels)

print("Subject Folders:", Scalogram_subject_folders)
print("Labels:", Scalogram_labels)

Subject Folders: ['/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_1'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_2'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_3'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_4'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_5'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_6'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_7'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_8'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_9'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_10'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_11'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_12'
 '/content/drive/MyDrive/Alzheimer/Scalogram Images/Temporal/Subject_13'
 '/content/drive/MyDrive/Alzheimer/Scalogra

In [ ]:
# Define a custom callback to start saving checkpoints from epoch 30
class CustomCheckpoint(ModelCheckpoint):
    def __init__(self, start_epoch, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.start_epoch = start_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch >= self.start_epoch:
            super().on_epoch_end(epoch, logs)

def train_and_evaluate_model(base_model, Scalogram_subject_folders, Scalogram_labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")

        model = create_new_model(base_model)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(Scalogram_subject_folders, Scalogram_labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)

        checkpoint_dir = '/content'
        os.makedirs(checkpoint_dir, exist_ok=True)

        local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'

        checkpoint_callback = CustomCheckpoint(
            start_epoch=30,
            filepath=local_checkpoint_filepath,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=True,
            verbose=1
        )

        class LearningRateLogger(Callback):
            def on_epoch_end(self, epoch, logs=None):
                logs = logs or {}
                logs['learning_rate'] = self.model.optimizer.learning_rate.numpy()
        lr_logger = LearningRateLogger()

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False, start_from_epoch=30)

        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=32, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping, lr_logger], verbose=1)

        if os.path.exists(local_checkpoint_filepath):
            print(f"Best weights saved locally at: {local_checkpoint_filepath}")

            shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
            if os.path.exists(drive_checkpoint_filepath):
                print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
            else:
                print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        else:
            print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")

        model.load_weights(local_checkpoint_filepath)

        test_predictions = model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        print("ModelCheckpoint callback triggered and saved the best weights.")

        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)

        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)

        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        files.download(file_name)


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx




    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")



In [ ]:
Inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [ ]:
model_name = 'InceptionV3_Temporal_Scalogram'
train_and_evaluate_model(Inception_base_model, Scalogram_subject_folders, Scalogram_labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:


## Model

In [ ]:
import os
import numpy as np
import math
import random
from collections import Counter
from google.colab import files
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, Model, clone_model
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, ConvLSTM2D,MaxPooling3D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, EfficientNetB2
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Input
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import shutil


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np


# Paths to the saved weights
spectrogram_model_path = '/content/InceptionV3_Temporal_Spectrogram_best_weights_iteration_1.h5'
scalogram_model_path = '/content/InceptionV3_Temporal_Scalogram_best_weights_iteration_1.h5'

# Function to build the model architecture
# def build_model():
#     base_model = InceptionV3(weights=None, include_top=False)
#     x = base_model.output
#     x = GlobalAveragePooling2D()(x)
#     return Model(inputs=base_model.input, outputs=x)

# Build the models
# inception_spectrogram = build_model()
# inception_scalogram = build_model()
from tensorflow.keras.models import model_from_json

# Load model architecture from the .json file
with open('/content/drive/MyDrive/Alzheimer/InceptionV3_Temporal_Spectrogram_iteration_1.json', 'r') as json_file:
    json_model = json_file.read()
inception_spectrogram = model_from_json(json_model)

# Load weights into the model
inception_spectrogram.load_weights('/content/drive/MyDrive/Alzheimer/InceptionV3_Temporal_Spectrogram_best_weights_iteration_1 (1).h5')



with open('/content/drive/MyDrive/Alzheimer/InceptionV3_Temporal_Scalogram_iteration_1.json', 'r') as json_file:
    json_model = json_file.read()
inception_spectrogram = model_from_json(json_model)

# Load weights into the model
inception_spectrogram.load_weights('/content/drive/MyDrive/Alzheimer/InceptionV3_Temporal_Scalogram_best_weights_iteration_1.h5')


# # Load the weights
# inception_spectrogram = load_model(spectrogram_model_path)
# inception_scalogram = load_model(scalogram_model_path)

# Function to create feature extractor
def create_feature_extractor(model):
    for layer in model.layers:
        layer.trainable = False
    output = model.layers[-1].output  # Use the last layer output
    return Model(inputs=model.input, outputs=output)

# Create feature extractors
feature_extractor_spectrogram = create_feature_extractor(inception_spectrogram)
feature_extractor_scalogram = create_feature_extractor(inception_scalogram)

# Assuming you already have loaded and preprocessed spectrogram and scalogram images
# Load and preprocess the images
train_images_spectrogram, train_labels, val_images_spectrogram, val_labels, test_images_spectrogram, test_labels = load_and_preprocess_data(Spectrogram_subject_folders, Spectrogram_labels)
train_images_scalogram, _, val_images_scalogram, _, test_images_scalogram, _ = load_and_preprocess_data(Scalogram_subject_folders, Scalogram_labels)

# Extract features
train_features_spectrogram = feature_extractor_spectrogram.predict(train_images_spectrogram)
val_features_spectrogram = feature_extractor_spectrogram.predict(val_images_spectrogram)
test_features_spectrogram = feature_extractor_spectrogram.predict(test_images_spectrogram)

train_features_scalogram = feature_extractor_scalogram.predict(train_images_scalogram)
val_features_scalogram = feature_extractor_scalogram.predict(val_images_scalogram)
test_features_scalogram = feature_extractor_scalogram.predict(test_images_scalogram)

# Concatenate features
train_features = np.concatenate((train_features_spectrogram, train_features_scalogram), axis=1)
val_features = np.concatenate((val_features_spectrogram, val_features_scalogram), axis=1)
test_features = np.concatenate((test_features_spectrogram, test_features_scalogram), axis=1)

# Build the final model
input_shape = train_features.shape[1]
input_layer = Input(shape=(input_shape,))
x = Dense(256, activation='relu')(input_layer)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_callback = ModelCheckpoint('best_model.h5', save_best_only=True)

history = model.fit(train_features, train_labels, epochs=50, batch_size=32, validation_data=(val_features, val_labels), callbacks=[early_stopping, checkpoint_callback])

# Evaluate the model
test_predictions = (model.predict(test_features) > 0.5).astype(int)
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions)
recall = recall_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions)

print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")


ValueError: too many values to unpack (expected 6)

In [ ]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_1128 (Conv2D)        (None, 149, 149, 32)         864       ['input_13[0][0]']            
                                                                                                  
 batch_normalization_1128 (  (None, 149, 149, 32)         96        ['conv2d_1128[0][0]']         
 BatchNormalization)                                                                              
                                                                                                  
 activation_1128 (Activatio  (None, 149, 149, 32)         0         ['batch_normalization_1